In [1]:
import tensorflow as tf
import glob
import cv2
import numpy as np
from config import *



def rep_data_gen():
    a = []
    for i in range(160):
        inst = car_images[i]
        img = cv2.imread(inst)
        img = cv2.resize(img, (94, 24))
        img = img / 255.0
        img = img.astype(np.float32)
        a.append(img)
    a = np.array(a)
    print(a.shape) # a is np array of 160 3D images
    img = tf.data.Dataset.from_tensor_slices(a).batch(1)
    for i in img.take(32):
        print(i)
        yield [i]

converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(recognition_edge_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from C:\Users\carlos\Desktop\cs\coral_tpu\models\recognition\variables\variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: inputs
INFO:tensorflow: tensor name: serving_default_inputs:0, shape: (-1, 24, 94, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: output_1
INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 24, 36), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from C:\Users\carlos\Desktop\cs\coral_tpu\models\recogniti

In [3]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_quant_model)